## Gaussian Process Classification

# Packages, sklearn

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, DotProduct, RationalQuadratic, Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import OneHotEncoder

# Retriving and processing the data

In [2]:
#choose a seed.
seed = 1
np.random.seed(seed)

# Import some data to play with.
digits = datasets.load_digits()

X = digits.data
y = np.array(digits.target, dtype = int)
X,y = shuffle(X,y)
N,d = X.shape

N = np.int64(600)
Ntrain = np.int64(500)
Ntest = np.int64(100)

Xtrain = X[0:Ntrain-1,:]
ytrain = y[0:Ntrain-1]
Xtest = X[Ntrain:N,:]
ytest = y[Ntrain:N]

kernels = {
    "RBF": 1.0 * RBF(length_scale=1.0),  # Isotropic RBF Kernel
    "DotProduct": DotProduct(sigma_0=10.0),
    "RationalQuadratic": 1.0 * RationalQuadratic(length_scale=1.0, alpha=1.0),
    "Matern": 1.0 * Matern(length_scale=1.0, nu=10),  # Nu can be 0.5, 1.5, or 2.5
    "Combined (RBF + DotProduct)": 1.0 * RBF(length_scale=1.0) + DotProduct(sigma_0=1.0),
}

# Part a - Gaussian Process classification (sklearn - several OvR classifiers)

In [4]:
for kernel_name, kernel in kernels.items():
    print(f"Testing kernel: {kernel_name}")
    gpc = GaussianProcessClassifier(kernel=kernel, optimizer=None)  # Disable hyperparameter optimization for consistency.
    gpc.fit(Xtrain, ytrain)
    
    # Training predictions.
    yp_train = gpc.predict(Xtrain)
    train_error_rate = np.mean(np.not_equal(yp_train, ytrain))
    
    # Test predictions.
    yp_test = gpc.predict(Xtest)
    test_error_rate = np.mean(np.not_equal(yp_test, ytest))
    
    # Print results.
    print(f"Training error rate for {kernel_name}: {train_error_rate}")
    print(f"Test error rate for {kernel_name}: {test_error_rate}\n")

Testing kernel: RBF
Training error rate for RBF: 0.0
Test error rate for RBF: 0.87

Testing kernel: DotProduct
Training error rate for DotProduct: 0.0
Test error rate for DotProduct: 0.14

Testing kernel: RationalQuadratic
Training error rate for RationalQuadratic: 0.0
Test error rate for RationalQuadratic: 0.12

Testing kernel: Matern
Training error rate for Matern: 0.0
Test error rate for Matern: 0.79

Testing kernel: Combined (RBF + DotProduct)
Training error rate for Combined (RBF + DotProduct): 0.0
Test error rate for Combined (RBF + DotProduct): 0.13



# Part b - Classification using Gaussian Process regressor

In [22]:
# Recode the labels as 1-hot representation. 
encoder = OneHotEncoder(sparse_output=False)  # sparse=False returns a dense array.
ytrain_reshaped = ytrain.reshape(-1, 1)
#print(ytrain_reshaped.shape) # Makes it into a 2D array.

y_train_one_hot = encoder.fit_transform(ytrain_reshaped)
#kernel = 1.0 * RBF([1.0]) # isotropic kernel
#kernel = Matern(length_scale=1, nu=2)
kernel = DotProduct(sigma_0=10.0)
#gpr_rbf = GaussianProcessRegressor(kernel=kernel).fit(Xtrain, y_train_one_hot)

gpr_rbf = GaussianProcessRegressor(kernel=kernel, optimizer=None)
gpr_rbf.fit(Xtrain, y_train_one_hot)

yp_train_one_hot = gpr_rbf.predict(Xtrain)
yp_train = encoder.inverse_transform(yp_train_one_hot)

# test_array = np.array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1]).reshape(1, -1)
# print(test_array.shape)
# test = encoder.inverse_transform(test_array)
# print(test)

train_error_rate = np.mean(np.not_equal(yp_train, ytrain))
print('Training error rate')
print(train_error_rate)

# yp_test_one_hot = gpr_rbf.predict(Xtest)
# yp_test = encoder.inverse_transform(yp_test_one_hot)
# test_error_rate = np.mean(np.not_equal(yp_test, ytest))
# print('Test error rate')
# print(test_error_rate)

Training error rate
0.899012453765246
